RSA

https://en.wikipedia.org/wiki/RSA_%28cryptosystem%29#Example

choose to prime numbers

In [49]:
#p, q = 7, 11
#p, q = 53, 61
p, q = 7907, 7919

In [50]:
#Berechne
n = p*q
phi_n = (p-1) * (q-1) # eulerscher Funktion: Anzahl aller Zahlen von 1-n, die teilerfremd (coprime) zu n sind (ggt(x, n) =1)

print(n)

62615533


In [51]:
# Satz von Euler: for all k,m with m<n it holds
# m^{k phi(n)+1}=m (mod n)

In [52]:
def euclid(x, y):
    if y>x:
        x,y = y,x
    if y==0:
        return x
    return euclid(x%y, y)

# Berechne first key: e with ggt(e, phi(n))=1 ==> inverse exist
for e in range(2, phi_n):
    if euclid(e, phi_n)==1:
        break
        
print(e)

3


In [53]:
# Berechne second key: d, with e*d = 1 mod n
# euklidsche algo: bestimme d mit e*d=1 (mod phi(n))

# https://cp-algorithms.com/algebra/extended-euclid-algorithm.html
def extended_euclid(a, b):
    if b>a:
        m, n, r = extended_euclid(b, a)
        return (n, m, r)
    if b==0:
        return (1, 0, a)
    m, n, r = extended_euclid(b, a%b)
    return n, m - n * (a//b), r
    
_, d, r = extended_euclid(phi_n, e)
assert r == 1, "d is not coprime to phi(n)"
d = d % phi_n
print(d)

# debug
#for i in range(60):
#    print(f'7*{i}%60={7*i%60}')

41733139


In [79]:
def fast_exp(base, exp, mod):
    if exp == 0:
        return 1
    if exp == 1:
        return base
    if exp & 1:
        return base * fast_exp(base ** 2 % mod, exp>>1, mod) % mod
    return fast_exp(base ** 2 % mod, exp>>1, mod) % mod

def encrpt(msg, public_key=e, n=n):
    return fast_exp(msg, public_key, n)

def decrypt(code, private_key=d, n=n):
    return fast_exp(code, private_key, n)

In [80]:
msg = 53
assert msg < n, f'message must be shorter than n={n}'
code = encrpt(msg)
print(f'encrypt({msg})={code}')

msg2 = decrypt(code)
print(f'decrypt({code})={msg2}')

encrypt(53)=148877
decrypt(148877)=53


In [81]:
# Encode Strings to Int

In [82]:
import numpy as np

def str2int(m, char_encoding_digits=3):
    return sum([
        ord(c)*10**(char_encoding_digits*(len(m)-i-1))
        for i, c in enumerate(m)
    ])

def int2str(c, char_encoding_digits=3):
    n_blocks = int(np.ceil(np.log10(c))/char_encoding_digits)
    return ''.join([
        chr((c//10**(char_encoding_digits * (n_blocks-i-1)))% 10**char_encoding_digits)
        for i in range(n_blocks)
    ])

m = 'hallo'
m_e = str2int(m)
m_d = int2str(m_e)

print(m)
print(m_e)
print(m_d)

hallo
104097108108111
hallo


In [84]:
m = 'ok'
msg = str2int(m)
assert msg < n, f'message {msg} must be shorter than n={n}'
print(f'str2int("{m}")={msg}')

code = encrpt(msg)
print(f'encrypt({msg})={code}')

msg2 = decrypt(code)
print(f'decrypt({code})={msg2}')
m2 = int2str(msg2)
print(f'int2str("{msg2}")={m2}')

int2str(decrypt(encrpt(str2int(m))))

str2int("ok")=111107
encrypt(111107)=48231358
decrypt(48231358)=111107
int2str("111107")=ok


'ok'